# O que é uma Rede Neural Artificial?

Uma rede neural artificial é um grafo composto por funções matemáticas como combinações lineares e funções de ativação. O grafo consiste de nós e arestas.

Nós em cada camada (exceto por nós na camada de entrada), realizam funções matemáticas usando como entrada a saída das camadas anteriores. Por exemplo, um nó pode representar `f(x,y)=x+y`, onde `x` e `y` são valores de entrada vindos da camada anterior.

De forma similar, cada nó cria uma saída que pode ser passada para os nós da próxima camada. O valor de saída da camada de saída não é passada para uma camada futura (pois é a última!).

Camadas entre a camada de entrada e a de saída são chamadas de camadas ocultas.

## Propagação para a Frente

Ao propagar valores da primeira cada (camada de entrada) através das funções matemáticas apresentadas em cada nó, a rede emite um valor de saída. Este processo é chamado de propagação para a frente, ou forward pass.

Aqui está um exemplo simples de propagação para a frente.

## Grafos

Os nós e arestas criam uma estrutura de grafo. Apesar de que o exemplo acima é razoavelmente simples, não é difícil imaginar que grafos crescentemente complexos sejam capazes de calcular... bem... qualquer qualquer coisa.

Existem geralmente dois passos para a criação de uma rede neural:

1. Definir o grafo de nós e arestas.
2. Propagar os valores dentro do grafo.

O `NanoFlow` trabalha da mesma forma. Você definirá os nós e arestas da sua rede com um método e irá propagar os valores dentro do grafo usando outro método. 

Sabemos que cada nó pode receber entradas de diversos outros nós. Sabemos também que cada nó cria uma saída única, que muito provavelmente será passada para outros nós. Vamos adicionar duas listas então: uma para adicionar referências aos nós de entrada, e outra para as referências dos nós de saída.

Sabemos que cada nó pode receber entradas de diversos outros nós. Sabemos também que cada nó cria uma saída única, que muito provavelmente será passada para outros nós. Vamos adicionar duas listas então: uma para adicionar referências aos nós de entrada, e outra para as referências dos nós de saída.

In [14]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Nó(s) que fornecem valores para o nó corrente. Referência aos nós de entrada.
        self.inbound_nodes = inbound_nodes
        # Nó(s) aos quais o nó corrente passa valores. Referência aos nós de saída.
        self.outbound_nodes = []
        # Para cada nó  aqui, adicionamos este nó como um nó de saída *daquele* nó.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # Cada nó eventualmente calculará um valor que representa a sua saída
        self.value = None
        
    # Cada nó será capaz de passar valores "forward propagation" e "back propagation".
    def forward(self):
        """
        Propagação para a frente.

        Calcula o valor de saída baseando-se nos `inbound_nodes` e
        armazena o valor final em self.value.
        """
        raise NotImplemented

### A subclasse Node
Diferentemente das outras subclasses de `Node`, a subclasse `Input` não calcula nada. A subclasse `Input` apenas armazena um valor `value`, como um dado de um atributo ou um parâmetro do modelo (peso/viés).

Você pode definir o valor `value` de forma explícita ou usando o método `forward()`. Este valor é usado para alimentar o restante da rede neural.

In [15]:
class Input(Node):
    # Pense em Input como o agrupamento de múltiplos nós de entrada individuais em um único Nó.
    def __init__(self):
        # Um nó Input não possui nós de entrada,
        # então não há necessidade de passar nada para o construtor.
        Node.__init__(self)

    # NOTA: O nó Input é o único onde o valor pode
    # ser passado como um argumento para forward().
    #
    # Todas as outras implementações de nós devem receber o valor
    # do nó anterior disponível em self.inbound_nodes
    #
    # Exemplo:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        # Sobre-escreva o valor se um valor foi fornecido.
        if value is not None:
            self.value = value

### A subclasse Add

Note a diferença no método `__init__: Add.__init__(self, [x, y])`. Diferentemente da classe `Input`, que não possui nós como entrada, a subclasse `Add` recebe dois nós no momento de sua criação, x e y, e adiciona os valores destes nós.


In [16]:
"""
Can you augment the Add class so that it accepts any number of nodes as input?

Hint: this may be useful:
https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists
"""
class Add(Node):
    # Utilizo o construtor para passar os "inbound_nodes" isnanciados na classe Node
    def __init__(self, x, y):
        Node.__init__(self, [x, y])
 
    def forward(self):
        x_value = self.inbound_nodes[0].value
        y_value = self.inbound_nodes[1].value
        self.value = x_value + y_value


NanoFlow possui dois métodos para te ajudar a definir e então executar valores dentro de seus grafos: **topological_sort()** e **forward_pass()**.

### topological_sort

Para definir a sua rede, você precisará definir a ordem das operações para os seus nós. Dado que a entrada de alguns nós depende da saída de outros nós, você precisará encadear o grafo de forma que as dependências das entradas de cada um dos nós estejam disponíveis antes de executar o seu cálculo. Esta técnica é chamada de [ordenação topológica](https://en.wikipedia.org/wiki/Topological_sorting).

A função `topological_sort()` implementa a ordenação topológica usando o [Algoritmo de Kahn](https://en.wikipedia.org/wiki/Topological_sorting#Kahn.27s_algorithm). Os detalhes deste método não são importantes neste momento, mas seu resultado é, `topological_sort()` retorna uma lista de nós em uma sequência que permite que todos os cálculos sejam realizados de forma serializada (em série). O método `topological_sort()` recebe como entrada um `feed_dict`, que é como nós inicializamos um valor para um nó `Input`. O `feed_dict` é representado na forma de um dicionário Python. **Aqui está um exemplo de uso:**

```python
# Define 2 nós do tipo "Input".
x, y = Input(), Input()
# Define um nó do tipo "Add" e usa os dois nós do tipo "Input" como entrada.
add = Add(x, y)
# Os valores de "x" e "y" serão definidos como 10 e 20, respectivamente.
feed_dict = {x: 10, y: 20}
# Ordena os nós usando ordenação topológica.
sorted_nodes = topological_sort(feed_dict=feed_dict)
```


In [17]:
def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L

### forward_pass

O outro método disponível é o **forward_pass()**, que de fato executa a rede e emite uma saída.

In [18]:
def forward_pass(output_node, sorted_nodes):   
    """
    Performs a forward pass through a list of sorted nodes.

    Arguments:
        `output_node`: A node in the graph, should be the output node (have no outgoing edges). O nó de saída do grafo (sem arestas de saída).
        `sorted_nodes`: A topologically sorted list of nodes. Uma lista topologicamente ordenada de nós.

    Returns the output Node's value
    """

    for n in sorted_nodes:
        n.forward()

    return output_node.value

In [25]:
def forward_pass_v2(graph):
    """
    Executa o forward pass através de uma lista de Nodes ordenados.

    Arguments:
        `graph`: The result of calling `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

### A subclasse Linear

As redes neurais recebem entradas e produzem saídas. Redes neurais podem melhorar a acurácia de sua saída com o tempo.

Um neurônio artificial simples depende de três componentes:

* entradas, xi
* pesos, wi
* viés, b

A saída, y, é a soma ponderada das entradas mais o viés.

Ao variar os pesos, você pode variar a influência que qualquer entrada tenha na saída. O aspecto de aprendizagem de redes neurais se dá durante um processo chamado retro-propagação (backpropagation). Na retro-propagação, a rede modifica os pesos para melhorar a acurácia da saída da rede. Você irá aplicar estes conceitos em breve.

A função Linear é um neurônio linear que gera uma saída ao aplicar uma versão simplificada de uma soma ponderada. A função Linear deve receber como entrada **n** nós de entrada, uma lista de pesos também com tamanho **n** e um viés (bias).

Álgebra linear reflete bem a idéia de transformar valores em camadas em um grafo. Existe uma técnica chamada **transformation** que realiza exatamente o que uma **camada** deve fazer - ela converte entradas em saídas em diversas dimensões. Observe a equação de saída:

![1](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5892a66c_neuron-output/neuron-output.png)

Durante os cáculos, vamos denotar **x** como **X** e **w** como **W** pois estes são agora matrizes. Neste cenário **b** será um vetor ao invés de um valor escalar.

Considere o nó **Linear** com uma entrada e **k** saídas (mapeando 1 entrada para k saídas). Neste contexto, uma entrada ou saída são sinônimos de atributos. Neste caso, **X** é uma matriz de 1 por 1.

![1](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f9571_newx/newx.png)

**W** se torna uma matriz de ordem 1 por **k** (se parece como uma linha).

![1](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f9571_neww/neww.png)

O resultado da multiplicação das matrizes **X** e **W** é uma matriz de ordem 1 por **k**. Como **b** também é uma matriz de 1 por **k**, **b** é adicionado diretamente a saída da multiplicação de **X** e **W**. E se quiséssemos mapear **n** entradas para **k** saídas?

Então **X** seria uma matriz de 1 por **n** e **W** uma matriz de **n** por **k**. O resultado desta multiplicação ainda seria uma matriz de 1 por **k**, então o uso dos viéses continuam os mesmos.

![1](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581f9570_newx-1n/newx-1n.png)
![1](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/58a24e51_neww-nk-fixed/neww-nk-fixed.gif)
![1](https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581a94e5_b-1byk/b-1byk.png)

Vamos ver agora um exemplo com **n** entradas. Considere uma imagem em escala de cinza de tamanho **28px** por **28px** (como o dataset MNIST). Nós podemos reformatar esta imagem para que ela se torne uma matriz de ordem **1 por 784**, ou seja, **n = 784**. Cada pixel é uma entrada/atributo.

Na prática, o padrão é usar várias entradas simultaneas durante a "passagem para a frente" de uma rede neural. Não é factível usar uma entrada por vez. O principal motivo para isto é que cada um dos exemplos pode ser processado em paralelo, resultando em grandes melhorias em performance. O número de entradas é chamado de **batch size**. Números comuns para este parâmetro são 32, 64, 128, 256 e 512. Normalmente, estes são os máximos que conseguimos comportar em memória. O que isso significa para **X**, **W** e **b**?

**X** se torna uma matriz de **m** por **n** enquanto **W** e **b** se mantém iguais. O resultado da multiplicação de matrizes é agora **m** por **k**, então a adioção de **b** é o resultado de um broadcast sobre cada linha.

In [19]:
class Linear(Node):
    #def __init__(self, inputs, weights, bias):
    def __init__(self, X, W, b):
        #Node.__init__(self, [inputs, weights, bias])
        Node.__init__(self, [X, W, b])

        # NOTE: The weights and bias properties here are not
        # numbers, but rather references to other nodes.
        # The weight and bias values are stored within the
        # respective nodes.

    def forward(self):
        """
        Set self.value to the value of the linear function output. y=∑wi xi + b
        
        Nesta solução, eu defini self.value como sendo o viés (bias) e então iterei sobre os valores de entrada 
        e pesos, adicionando cada entrada já ponderada em self.value. Note que chamar .value em self.inbound_nodes[0] 
        ou self.inbound_nodes[1] nos retorna uma lista.
        """
        #inputs = self.inbound_nodes[0].value
        #weights = self.inbound_nodes[1].value
        #bias = self.inbound_nodes[2]
        #self.value = bias.value
        #for x, w in zip(inputs, weights):
        #    self.value += x * w
            
        """
        Na solução v2, estou preparando o NanoFlow para trabalhar com matrizes e vetores.
        Você precisará usar o método np.dot, que trabalha como uma multiplicação de matrizes 2D, para multiplicar as matrizes de entrada e pesos da Equação (2).
        É importante também notar que o numpy, na prática, sobrecarrega o operador __add__ para que você possa usá-lo diretamente com estruturas do tipo np.array (eg. np.array() + np.array()).
        Obtive os valores de X, W e b das suas respectivas entradas. Eu usei o método np.dot para realizar a multiplicação de matrizes.
        """
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        self.value = np.dot(X, W) + b


### Sigmoid

Redes neurais tiram vantagem de diferentes transformações e funções de ativação para melhorar as suas saídas. **Transformações lineares** são ótimas para aplicar desvios a valores, mas redes neurais normalmente requerem transformações mais complexas. Por exemplo, um dos primeiros projetos de neurônio artificial, o [perceptron](https://en.wikipedia.org/wiki/Perceptron), exibia um comportamento de saída binário. Perceptrons comparam uma entrada ponderada com um limiar. QUando a saída excede este limiar, o perceptron é ativado e a saída se torna 1, ou em caso contrário, 0.

A idéia do comportamento binário geralmente faz sentido para problemas de classificação. Por exemplo, se você pedir para uma rede hipotetizar se uma imagem manuscrita é um '9', estamos falando de uma saída binária - sim, é um **'9'**, ou não, este não é um **'9'**. No caso do perceptron podemos usar a **step function** ou **função degrau** que é uma das formas mais simples de saída binária. O problema é que este tipo de função não é contínua e diferenciável, o que é muito ruim, já que a **diferenciação** é o que torna o processo de **descida do gradiente** possível.

![sigmoid](https://d17h27t6h515a5.cloudfront.net/topher/2016/October/58102c91_19/19.png)

Já a **função sigmoide**, apresentada na equação acima, substitui a limiarização com uma bela curva em formato de **S**, que funciona como uma limiarização, mas é capaz de manter a continuidade e também a diferenciação. Como um bônus, a função sigmoide tem uma derivada que é muito parecida com a própria sigmoide original.

Note que a função sigmoide possui apenas um parâmetro. Lembre-se que a sigmoide é uma função de ativação (não linear), o que significa que ela recebe uma entrada única e realiza uma operação matemática sobre este valor.

Conceitualmente, a função sigmoide toma decisões. Quando alimentada com atributos ponderados sobre dados, essa função indica se os atributos contribuem ou não para a classificação. Deste modo, uma sigmoide trabalha bem seguida de uma função linear. Até o momento, usando pesos e viéses aleatórios, a saída de um nó sigmoide é também aleatório. O processo de aprendizagem usando retro-propagação e descida do gradiente, modifica os pesos e viéses, e com isso, a saída da ativação começará a aderir às saídas esperadas.

In [20]:
class Sigmoid(Node):
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, x):
        """
        Este método está isolado da função "forward" pois será usada na função "backward" também.

        "x": Um objeto em formato de array numpy.
        """
        return 1. / (1. + np.exp(-x)) # o `.` garante que `1` é um float

    def forward(self):
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)

### Mean Squared Error

Como você deve lembrar, redes neurais melhoram a acurácia de suas saídas ao modificar os pesos e viéses das suas saídas em resposta ao dados rotulados de treinamento.

Existem diferentes técnicas para definir a acurácia de uma rede neural, sendo que todas estão focadas na habilidade das redes em aproximar as suas saídas às saídas esperadas. Diferentes métricas são usadas para medir a acurácia, normalmente sendo denominadas como **perda (loss)** ou **custo (cost)**.

![mse](https://d17h27t6h515a5.cloudfront.net/topher/2016/December/585ab25f_codecogseqn/codecogseqn.png)

Aqui, w denota a coleção de todos os pesos na rede, b todos os viéses, m é o número de exemplos de treinamento, a é uma aproximação de y(x) dada pela rede, sendo que a e y(x) são vetores de mesmo tamanho.

A coleção de pesos são todas as matrizes de peso vetorizadas e concatenadas em um grande vetor. O mesmo ocorre para a coleção de viéses exceto pelo fato de que elas já são vetores e não existe necessidade de vetorizá-las, mas apenas concatená-las. Aqui está um exemplo da criação de w em código:

```python
# matrizes 2 por 2
w1  = np.array([[1, 2], [3, 4]])
w2  = np.array([[5, 6], [7, 8]])

# vetorização
w1_flat = np.reshape(w1, -1)
w2_flat = np.reshape(w2, -1)

w = np.concatenate((w1_flat, w2_flat))
# array([1, 2, 3, 4, 5, 6, 7, 8])
```

A matemática por trás do MSE reflete acima, onde **y** é a saída desejada e **a** é a saída computada pela rede neural. Nós então calculamos o quadrado da diferença `diff**2` (ou alternativamente np.square(diff)). Por fim, nós precisamos somar os quadrados das diferenças e dividir pelo número total de exemplos m. Isto pode ser feito com a função `np.mean` ou calculando `(1 /m) * np.sum(diff**2)`.

Esta é uma maneira interessante de abstrair todos os pesos e viéses usados na rede neural e torna as coisas mais simples de se escrever como veremos em breve nas seções sobre descida do gradiente.

O custo, `C`, depende da diferença entre a saída correta, `y(x)`, e a saída fornecida pela rede, `a`. É fácil verificar que não há diferença entre `y(x)` e `a` (para todos os valores de x), o que nos leva a um custo `0`.

Esta é uma situação ideal, e o processo de aprendizagem tenta minimizar o custo quanto for possível.

In [21]:
class MSE(Node):
    def __init__(self, y, a):
        """
        The mean squared error cost function.
        Should be used as the last node for a network.
        """
        # Call the base class' constructor.
        Node.__init__(self, [y, a])

    def forward(self):
        """
        Calculates the mean squared error.
        """
        # NOTE: We reshape these to avoid possible matrix/vector broadcast
        # errors.
        #
        # For example, if we subtract an array of shape (3,) from an array of shape
        # (3,1) we get an array of shape(3,3) as the result when we want
        # an array of shape (3,1) instead.
        #
        # Making both arrays (3,1) insures the result is (3,1) and does
        # an elementwise subtraction as expected.
        y = self.inbound_nodes[0].value.reshape(-1, 1)
        a = self.inbound_nodes[1].value.reshape(-1, 1)
        m = self.inbound_nodes[0].value.shape[0]

        diff = y - a
        self.value = np.mean(diff**2)

### Linear Transformation

In [22]:
import numpy as np

X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(f, graph)

"""
Output should be:
[[-9., 4.],
[-9., 4.]]
"""
print(output)

[[-9.  4.]
 [-9.  4.]]


### Sigmoid

In [23]:
X, W, b = Input(), Input(), Input()

f = Linear(X, W, b)
g = Sigmoid(f)

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

feed_dict = {X: X_, W: W_, b: b_}

graph = topological_sort(feed_dict)
output = forward_pass(g, graph)

"""
Output should be:
[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]
"""
print(output)

[[  1.23394576e-04   9.82013790e-01]
 [  1.23394576e-04   9.82013790e-01]]


### MSE

In [26]:
y, a = Input(), Input()
cost = MSE(y, a)

y_ = np.array([1, 2, 3])
a_ = np.array([4.5, 5, 10])

feed_dict = {y: y_, a: a_}
graph = topological_sort(feed_dict)
# forward pass
forward_pass_v2(graph)

"""
Expected output

23.4166666667
"""
print(cost.value)

23.4166666667
